In [ ]:
import pulp
import time
from pulp import GUROBI
from haversine import haversine
from scipy.spatial.distance import squareform, pdist
import pandas as pd
sizes = [10,20,30,40,50,70,100]

#Create dictionary of decision variables and models
x = {}
t = {}
tsp = {}

#Create dataframe of results
result = pd.DataFrame(columns = ('No. of cities','Route Distance','Solution Time'))

#Create dataset, model and Solve for different No. of cities(sizes)
for size in sizes:
    coord = pd.read_csv('tsp_input.csv')[:size]
    coord.set_index('Place_Name', inplace=True)
    dis_mat = pd.DataFrame(squareform(pdist(coord, metric=haversine)), index=coord.index, columns=coord.index)
    edges = [(i,j) for i in coord.index for j in coord.index]
    x[size] = {}

    #Create decision variable
    for e in edges:
        x[size][e] = pulp.LpVariable("x_" + str(size) + '_' + str(edges.index(e)), cat='Binary')
    tsp[size] = pulp.LpProblem("TSP_size"+str(size),pulp.LpMinimize)
    #Define objective function
    z = 0
    for e in edges:
        z += x[size][e]*dis_mat.loc[e[0]][e[1]]
    tsp[size] += z
    t[size] = {}
    for i in coord.index:
        #Define Vertex sequence(subtour elimination) variable
        t[size][i] = pulp.LpVariable("t_" + str(size) + '_' + str(i),cat = 'Integer',lowBound = 1,upBound=size)
        #Define constraints
        tsp[size] += x[size][i,i]==0
        tsp[size] += pulp.lpSum(x[size][i,j] for j in coord.index) == 1
        tsp[size] += pulp.lpSum(x[size][j,i] for j in coord.index) == 1

    #Define subtour elimination constraint
    for i in coord.index:
        for j in coord.index:
            if(i!=j and list(coord.index).index(i)!=0 and list(coord.index).index(j)!=0):
                tsp[size] += t[size][j]>=t[size][i] + 1 - 2*size*(1-x[size][i,j])
    tsp[size].writeLP('tsp_puLP_size' + str(size) + '.lp')
    #Solve
    solver_start_time = time.time()
    tsp[size].solve(GUROBI())
    solver_end_time = time.time()
    #Add Route distance and Solution time to results dataframe
    result.loc[len(result['No. of cities'])] = [size ,round(z.value(),2),round(solver_end_time - solver_start_time, 2)]
    print(result)
result.to_csv('MIP(Gurobi)_TSP_results.csv')



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 102 rows, 109 columns and 426 nonzeros
Model fingerprint: 0x6c69d293
Variable types: 0 continuous, 109 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 7724.6612411
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 92 rows, 99 columns, 396 nonzeros
Variable types: 0 continuous, 99 integer (90 binary)

Root relaxation: objective 3.730832e+03, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3730.83197

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel\iostream.py", line 526, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 592789 74705     cutoff   69      9374.58839 9295.72220  0.84%  15.9 1281s
 596072 73697 9314.33259   94   77 9374.58839 9296.57219  0.83%  15.9 1287s
 597541 73248 9302.48434   73   93 9374.58839 9296.99531  0.83%  15.9 1290s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\ipykernel\iostream.py", line 526, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 603967 71333 9330.22247   74  102 9374.58839 9298.82278  0.81%  15.9 1302s
